In [ ]:
#Import Libraries

import graphviz
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import sklearn

from pylab import rcParams
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import scale

import seaborn as sb
sb.set_style('whitegrid')
from collections import Counter

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
import sklearn.metrics as sm
from sklearn.ensemble import RandomForestRegressor

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost
from sklearn.svm import SVR
from sklearn import ensemble

import math
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import KFold 
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from pyearth import Earth

In [ ]:
#Load the Data Set

data = pd.read_csv ("\\InputFolderName\\Scenario and ANO Data Combined.csv" )
subset3 = data.drop(['Column1','Year', 'gcm.1',	'Scenario_Year.1',	'Year.1', 'scenarios',	'Scenario Number'	,'Global Outlook'	,'GCM',	'Productivity'	,'Land Use Policy',	'Hurdle Rate',	'Constraint Setting',	'Scenario_Year'	,'Index_Value', 'capCons',	'hurdleRate'	,'climate'], axis= 1)

#Process and Prepare the Data

#Preapre Data for the Model
subset4 = data[['Column1','Year', 'gcm.1',	'Scenario_Year.1',	'Year.1', 'scenarios',	'Scenario Number'	,'Global Outlook'	,'GCM',	'Productivity'	,'Land Use Policy',	'Hurdle Rate',	'Constraint Setting',	'Scenario_Year'	,'Index_Value', 'capCons',	'hurdleRate'	,'climate']]

data4 = data[['Hurdle Rate', 'Constraint Setting']]

cleanup_nums = {
                "Hurdle Rate": {"5x": 5 , "2x": 2, "1x": 1 },
                "Constraint Setting":{"U":0, "C":1},
                }

obj_df2 = data4.replace(cleanup_nums)
oe_style = OneHotEncoder()
gcm = oe_style.fit_transform(subset4[['gcm.1']])
GCM = pd.DataFrame(gcm.toarray(), columns=oe_style.categories_)
climate = oe_style.fit_transform(subset4[['climate']])
Climate = pd.DataFrame(climate.toarray(), columns=oe_style.categories_)
# subset = subset.join(CS)
data4 = pd.concat([obj_df2,GCM, Climate], axis=1)
data4.columns = ['Hurdle Rate', 'Constraint Setting', 'CE2', 'MPI', 'MR5', 'H3', 'L1', 'M2', 'M3' ]
fromatted_data = pd.concat([subset3, data4], axis=1)

# the min-max approach (often called normalization) rescales the feature to a fixed range of [0,1] by subtracting the minimum value of the feature and then dividing by the range.
# create a scaler object
scaler = MinMaxScaler()
# fit and transform the data
df_norm2 = pd.DataFrame(scaler.fit_transform(fromatted_data), columns=fromatted_data.columns)
df_norm2.drop(['bioConstraintMax', 'yieldincreaseB','yieldincreaseS','yieldincreaseD','yieldincreaseI','inputincrease','diselpricepath'], axis='columns', inplace=True)


In [ ]:
#Detailed Final Model with all metrices
from sklearn import model_selection
import HydroErr as he
import math
def run_models(X_train , y_train, X_test, y_test) :
    dfs = []
    models = [
          ('LinearReg', LinearRegression()),
          
          ('SVR',SVR(kernel = 'poly', C = 3, degree = 8, gamma =  0.2)),
          ('GBRT', ensemble.GradientBoostingRegressor(n_estimators= 500,max_depth= 4, min_samples_split = 5, learning_rate = 0.01,loss= 'ls')),
          ('RF', RandomForestRegressor(random_state = 42)),
          ('XGB',  xgboost.XGBRegressor(n_estimators=200, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)),
           ('MARS',Earth() ),
           ('BayesianRidge', BayesianRidge(compute_score=True)),
           ('KernelRidge', KernelRidge(alpha=1.0))
             
        ]
    results = []
    names = []
    K=0
    scoring = ['r2', 'neg_mean_absolute_error', 'neg_root_mean_squared_error', 'explained_variance']
    for name, model in models:
        kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=42)
        cv_results = model_selection.cross_validate(model, X_train, y_train, cv=kfold, scoring=scoring, return_train_score=True)
        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        sim=y_pred
        obs=y_test
        MAE= np.round(he.mae(sim, obs),4)#Mean Absolute Error
        MSE= np.round(he.mse(sim, obs),4)#Mean Squared Error
        RMSE=np.round(he.rmse(sim, obs),4)#Root Mean Squared Error
        MLE=np.round(he.mle(sim, obs),4)#The mean log error value
        MALE=np.round(he.male(sim, obs), 4)#The mean absolute log error value.
        MSLE=np.round(he.msle(sim, obs), 4)#mean squared log error (MSLE)
        MDE=np.round(he.mde(sim, obs),4)#The median error value.
        MDAE=np.round(he.mdae(sim, obs),4)#The median absolute error value.
        MDSE=np.round(he.mdse(sim, obs),4)#The median squared error value.
        ED=np.round(he.ed(sim, obs),4)#The euclidean distance error value.
        NED=np.round(he.ned(sim, obs),4)#The normalized euclidean distance value.
        RMSLE=np.round(he.rmsle(sim, obs), 4)#The root mean square log error value.

        NRMSEV=np.round(he.nrmse_range(sim, obs),4)#The range normalized root mean square error value.
        NRMSE=np.round(he.nrmse_mean(sim, obs),4)#mean normalized root mean square error.
        MASE=np.round(he.mase(sim, obs),4)#mean absolute scaled error.
        r=np.round(he.pearson_r(sim, obs),4)#Pearson correlation coefficient.
        r2=np.round(he.r_squared(sim, obs),4)#Coefficient of Determination (r2).
        ACC=np.round(he.acc(sim, obs),4)#The anomaly correlation coefficient.
        MAPE=np.round(he.mape(sim, obs),4)#The mean absolute percentage error.
        MAPD=np.round(he.mapd(sim, obs),4)#The mean absolute percentage deviation.
        d=np.round(he.d(sim, obs),4)#Compute the the index of agreement (d).
        d1=np.round(he.d1(sim, obs),4)#The index of agreement (d1).
        drel=np.round(he.drel(sim, obs),4)#The relative index of agreement.
        MBR=np.round(he.mb_r(sim, obs),4)#The Mielke-Berry R value.
        NSE=np.round(he.nse(sim, obs),4)#NSE
        MNSE=np.round(he.nse_mod(sim, obs),4)#modified Nash-Sutcliffe efficiency value.
        RNSE=np.round(he.nse_rel(sim, obs),4)#relative Nash-Sutcliffe efficiency (NSE rel)
        KGE1=np.round(he.kge_2009(sim, obs),4)#the Kling-Gupta efficiency (2009).
        KGE=np.round(he.kge_2012(sim, obs),4)#Kling-Gupta (2012) efficiency
        LM=np.round(he.lm_index(sim, obs),4)#the Legate-McCabe Efficiency Index
        LME=np.round(he.d1_p(sim, obs),4)#The Legate-McCabe Efficiency 
        results.append(cv_results)
        names.append(name)
        this_df = pd.DataFrame(cv_results)
        this_df['model'] = name
        this_df['variable'] = y_train.name
        this_df['MAE']  = MAE
        this_df['RMSE']  = RMSE
        this_df['MLE']  = MLE
        this_df['MALE']  = MALE
        this_df['MSLE']  = MSLE
        this_df['MDE']  = MDE
        this_df['MDAE']  = MDAE
        this_df['MDSE']  = MDSE
        this_df['ED']  = ED

        this_df['NED']  =  NED
        this_df['RMSLE']  = RMSLE
        this_df[' NRMSEV']  =  NRMSEV
        this_df['NRMSE']  = NRMSE
        this_df['MASE']  = MASE

        this_df['r']  = r
        this_df['r2']  = r2
        this_df['ACC']  = ACC
        this_df['MAPE']  = MAPE
        this_df['MAPD']  = MAPD
        this_df['d']  = d
        this_df['d1']  = d1
        this_df['drel']  = drel

        this_df['MBR']  = MBR
        this_df['NSE']  = NSE
        this_df['MNSE']  = MNSE
        this_df['RNSE']  = RNSE
        this_df['KGE1']  = KGE1
        this_df['KGE']  = KGE
        this_df['LM']  = LM
        this_df['LME']  = LME

        dfs.append(this_df)
        final = pd.concat(dfs, ignore_index=True)
        
    return final.to_csv("\\OutputFolderName\\"+str(k)+"_results.csv")

In [ ]:
for k in range(0, 70):
    data = df_norm2[df_norm2.columns[k]]
    X_train, X_test, y_train, y_test = train_test_split(df_norm2.drop(df_norm2.columns[0:70], axis = 1), df_norm2[df_norm2.columns[k]] , train_size=0.8,test_size = 0.2, random_state = 1000)
    run_models(X_train, y_train, X_test, y_test)


In [ ]:
import os
import glob
import pandas as pd
os.chdir("\\OutputFolderName")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "OutputFolderName\\Detailed_Model_Run_Results.csv", index=False, encoding='utf-8-sig')

In [ ]:
#Sample upto Year 2050
# Latin HyperCube Generated Sample Data (1000 for first 9 variables then rest other variables are used as present of absent)
#this uniqu combination gives about 24 combinations thus the sample size becoex 10000*24 = 240000

from pyDOE import lhs
from scipy.stats.distributions import norm

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
# Specify random sample number
sample_num = 10000

# Using Latin Hypercube method to generate uniform distribution
uni_samples = lhs(n=17, samples=sample_num, criterion='maximin')

# Create the dataframe
df = pd.DataFrame(uni_samples)
df.columns = [
'yieldincreaseC', 
'yieldincreaseT', 
'agprice_path', 
'isprice_path',
'carbonpricepath', 
'petrolpricepath', 
'electricitypricepath',
'biodivFundCP', 
'Hurdle Rate', 
'Constraint Setting', 
'CE2', 
'MPI',
'MR5', 
'H3', 
'L1', 
'M2', 
'M3'
]

df.to_csv("\\OutputFolder\\TestRunData.csv")


In [ ]:
#Training the Best model out of all the models and producing the results based on the Hypercube samples

#Testing the model on output SENSITIVITY f0r 2050

#ROC Curve for Sensitivity Analaysis of Classification related models.
#Importing Required Modules
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from  sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve


for k in range(0, 70):
    test_data = pd.read_csv ("\\OutputFolder\\AllTestData_2050.csv" )
    P_train, P_test, Q_train, Q_test = train_test_split(test_data.drop(test_data.columns[0:1], axis = 1) , test_data[test_data.columns[0]] , train_size=0.99999,test_size = 0.00001, random_state = 45)

    #Splitting the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(df_norm2.drop(df_norm2.columns[0:70], axis = 1), df_norm2[df_norm2.columns[k]] , train_size=0.8,test_size = 0.2, random_state = 500)

    #Creating the class object and 
    model = xgboost.XGBRegressor(n_estimators=200, learning_rate=0.08, gamma=0, subsample=0.75,
                               colsample_bytree=1, max_depth=7)
    model.fit(X_train, y_train)

    #predict probabilities
    probs = model.predict(P_train)
    df = pd.DataFrame(P_train)
    df['Predicted'] = probs
    df['Setting'] = test_data[test_data.columns[0]]
#     df['Index'] = test_data[test_data.columns[1]]
    df['Prediction_For'] = df_norm2.columns[k]
    df.to_csv("\\OutputFolder\\SensitivityTest\\"+str(k)+"_25_03_2022Sensivtivity_results.csv")

In [ ]:
import os
import glob
import pandas as pd
os.chdir("\\OutputFolder\\SensitivityTest")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "\\OutputFolder\\SensitivityTest\\All_Sensitivity_Results.csv", index=False, encoding='utf-8-sig')

In [ ]:
All_SensitivityData = pd.read_csv("\\OutputFolder\\SensitivityTest\\SensitivityTest\\All_Sensitivity_Results.csv")
Pivot_Test = pd.DataFrame(pd.pivot_table(All_SensitivityData, values='Predicted', index=['Unnamed: 0','Setting'],
                    columns=['Prediction_For']))

Pivot_Test['Index'] = Pivot_Test.index.to_numpy()

# Pivot_Test.to_csv( "\\\mdsha.homes.deakin.edu.au\\my-home\\UserData\\Desktop\\PhD\\Year 1\\Data\\OutPut\\Scenario Analysis Test Final Run 17.03.2022\\SensitivityTest\\TestAll_Sensitivity_Results.csv", index=False, encoding='utf-8-sig')